# PySpark Tutorial - Applications
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

In [ ]:
from pyspark import SparkContext, SparkConf
import numpy as np
import operator

In [ ]:
conf=SparkConf().setAppName("pyspark tutorial").setMaster("local[*]")
sc = SparkContext(conf=conf)

## WordCount

In [ ]:
lines=sc.textFile("hamlet.txt")

In [ ]:
lines.take(3)

In [ ]:
counts = lines.flatMap(lambda line: line.split())\
              .map(lambda word: (word,1))\
              .reduceByKey(operator.add)

In [ ]:
counts.take(5)

In [ ]:
lines.flatMap(lambda line: line.split())\
              .map(lambda word: word.lower())\
              .map(lambda word: (word,1))\
              .reduceByKey(operator.add)\
              .sortBy(lambda x: x[1], ascending=False)\
              .take(5)

## Page Rank

We represent our graph as a simple vertex-edge-list with the edges stored as tuples. Because each node is a Key-Value, we can directly parallelize the graph and then operate on it using K-V operation.s

In [ ]:
graph = sc.parallelize([
    ('A', ('B')),
    ('B', ('A', 'C')),
    ('C', ('A', 'D')),
    ('D', ('A'))
])

The current page rank is represented as pairs of the node name the current value. We initialize the page rank to 1.0

In [ ]:
ranks = graph.map( lambda node: (node[0], 1.0))

In [ ]:
ranks.collect()

The current page will contribute its current rank divided by the number of out edges to each node. Because the edge list indicates the destination node, this will produce pairs of values indicating the node and the contribution

In [ ]:
def computeContrib(edges, rank):
    return ( (e, rank/len(edges)) for e in edges )

In [ ]:
list(computeContrib(('A','D'), 1.0))

We need to use both the graph and the current rank information -- we do this using a `join`

In [ ]:
graph.join(ranks).collect()

Now, to compute the contribution of each link for each node, we use use `computeContrib` for that nodes information (edge list & rank). Here's an example of that happening in a single step:

In [ ]:
graph.join(ranks).flatMap(lambda node: computeContrib(node[1][0],node[1][1])).collect()

Now, we reduce the values by the key and sum up the contributions. For example,

In [ ]:
graph.join(ranks).flatMap(lambda node: computeContrib(node[1][0],node[1][1]))\
    .reduceByKey(operator.add).collect()

These contributions are used to calculate the final page rank.

We can then perform the rank update operation multiple times until we converge to an answer. In our case, we're going to just run the code 5 times.

In [ ]:
for itr in range(5):
    print("=== Iteration {} ====".format(itr))
    contribs = graph.join(ranks).\
       flatMap(lambda node: computeContrib(node[1][0], node[1][1]))
    print("Contribs are", contribs.collect())
    ranks = contribs.reduceByKey(operator.add).\
                     mapValues(lambda rank: rank * 0.85 + 0.15)
    print("Ranks are", ranks.collect())
print("====")
print("Final rank:", ranks.collect())